In [1]:
! poetry install

Installing dependencies from lock file

No dependencies to install or update

Installing the current project: dialogue2graph (0.1.1)Installing the current project: dialogue2graph (0.1.1)


## Generate 3 augmented dialogues on one given dialogue

Use DialogueAugmenter to augment an original dialogue by paraphrasing its lines while maintaining the structure and flow of the conversation.

In [1]:
import json
import dotenv
from dialogue2graph.pipelines.core.dialogue import Dialogue
from dialogue2graph.datasets.augment_dialogues.augmentation import DialogueAugmenter
from dialogue2graph.pipelines.model_storage import ModelStorage
from dialogue2graph.datasets.augment_dialogues.prompts import augmentation_prompt_3_vars
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

/home/olshevskaya/miniconda3/envs/conda_env_py311/lib/python3.11/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "validate" in "Dialogue" shadows an attribute in parent "BaseModel"
  warnings.warn(
2025-04-30 11:32:04,076 - datasets - INFO - PyTorch version 2.6.0 available.


True

1. Read the example data to get the dialogue for augmentation.

In [2]:
with open("dialogue_augmentation_data_example.json", encoding="utf-8") as f:
    data = json.load(f)
dialogue = Dialogue(**data[0]["dialogue"])
print(dialogue)

assistant: Hi there! How can I assist you today?
user: I want to place an order.
assistant: Great! What product are you interested in?
user: I’d like a t-shirt.
assistant: What size would you like?
user: Medium, please.
assistant: Would you like that in red or blue?
user: Red, please.
assistant: Perfect! Can I have your shipping address?
user: Sure, it’s 123 Main St.
assistant: Thank you! Your order has been placed. Is there anything else I can help you with?
user: Actually, can I change my order?
assistant: Of course! What would you like to change?
user: I want a large size.
assistant: What size would you like?
user: Medium, please.
assistant: Would you like that in red or blue?
user: Red, please.
assistant: Perfect! Can I have your shipping address?
user: Sure, it’s 123 Main St.
assistant: Thank you! Your order has been placed. Is there anything else I can help you with?
user: No, that's all. Thank you.
assistant: Alright, feel free to reach out anytime. Have a great day!


2. Create `ModelStorage` instance and add choosen LLMs for dialogue generation (i.e. dialogue augmentation) and formatting LLM's output.

In [3]:
model_storage = ModelStorage()
model_storage.add(
        key="generation-llm", # model to use for generation
        config={"model_name": "gpt-4o-mini-2024-07-18", "temperature": 0.7},
        model_type=ChatOpenAI,
    )
model_storage.add(
        key="formatting-llm", # model to use for formatting LLM's output
        config={"model_name": "gpt-3.5-turbo", "temperature": 0.7},
        model_type=ChatOpenAI,
    )

2025-04-30 11:32:08,746 - /cephfs/home/olshevskaya/chatsky-llm-autoconfig/dialogue2graph/pipelines/model_storage.py - INFO - Added <class 'langchain_openai.chat_models.base.ChatOpenAI'> model 'generation-llm' to storage
2025-04-30 11:32:08,813 - /cephfs/home/olshevskaya/chatsky-llm-autoconfig/dialogue2graph/pipelines/model_storage.py - INFO - Added <class 'langchain_openai.chat_models.base.ChatOpenAI'> model 'formatting-llm' to storage


3. Create `DialogueAugmenter` instance and use `invoke` method to get augmented dialogues.

In [4]:
augmenter = DialogueAugmenter(
        model_storage=model_storage,
        generation_llm="generation-llm",
        formatting_llm="formatting-llm",
    )

In [5]:
result = augmenter.invoke(
        dialogue=dialogue,
        topic="Responding to DMs on Instagram/Facebook.",
        prompt=augmentation_prompt_3_vars,
    )

2025-04-30 11:32:44,258 - httpx - INFO - HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [6]:
result

[Dialogue(messages=[DialogueMessage(text='Hello! How can I help you today?', participant='assistant'), DialogueMessage(text="I'd like to make an order.", participant='user'), DialogueMessage(text='Awesome! Which product are you interested in?', participant='assistant'), DialogueMessage(text='I’m interested in a t-shirt.', participant='user'), DialogueMessage(text='What size do you need?', participant='assistant'), DialogueMessage(text='I’d like a medium size, please.', participant='user'), DialogueMessage(text='Would you prefer that in red or blue?', participant='assistant'), DialogueMessage(text='Red would be great, thank you.', participant='user'), DialogueMessage(text='Great choice! Could you provide your shipping address?', participant='assistant'), DialogueMessage(text="It's 123 Main St.", participant='user'), DialogueMessage(text='Thank you! Your order is confirmed. Is there anything else you need?', participant='assistant'), DialogueMessage(text='Is it possible to change my orde

**Example of augmented dialogue**

In [7]:
print(result[0])

assistant: Hello! How can I help you today?
user: I'd like to make an order.
assistant: Awesome! Which product are you interested in?
user: I’m interested in a t-shirt.
assistant: What size do you need?
user: I’d like a medium size, please.
assistant: Would you prefer that in red or blue?
user: Red would be great, thank you.
assistant: Great choice! Could you provide your shipping address?
user: It's 123 Main St.
assistant: Thank you! Your order is confirmed. Is there anything else you need?
user: Is it possible to change my order?
assistant: Absolutely! What would you like to change?
user: I need a large size instead.
assistant: What size would you like now?
user: I want medium again, please.
assistant: Would you like that in red or blue?
user: Red, please.
assistant: Great! Can you provide your shipping address once more?
user: It’s still 123 Main St.
assistant: Thank you! Your order has been successfully placed. Is there anything else I can assist you with?
user: No, that's all for 